In [1]:
from joblib import load
svm_best_model = load("/Users/jacquessham/Documents/MSDS/MSAN621/FinalProject/Dataset/svm_full.joblib")

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LinearSVC from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator Pipeline from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid result

First split the data

In [116]:
import pandas as pd

# Comments File
comments = '/Users/jacquessham/Documents/MSDS/MSAN621/FinalProject/Dataset/combined2.csv'
comm = pd.read_csv(comments)
filt = (comm.sectionName != "Unknown") & (comm.commentType == "comment")
unknown = (comm.sectionName == "Unknown") & (comm.commentType == "comment")
# Data that is known section, for training
data = comm.commentBody[filt]
sections = comm.sectionName[filt]
# Data that is unknown, for prediction
data_unknown = comm.commentBody[unknown]
sections_unknown = comm.sectionName[unknown]

In [134]:
sections_pd = pd.DataFrame(sections)
sections_pd.columns = ['Training_count']
rows_training_count = sections_pd.shape[0]
training_spread = sections_pd['Training_count'].value_counts()

def get_percentage_training(num):
    return float(num)/rows_training_count

In [135]:
# This is the report for training data
training_spread = pd.DataFrame(training_spread)

In [136]:
training_spread['Percent_training'] = training_spread.Training_count.apply(get_percentage_training)
training_spread['Index'] = training_spread.index

In [124]:
# This is the prediction for data that is in unknown section
results = svm_best_model.predict(data_unknown)

In [126]:
output = pd.concat([data_unknown.reset_index(drop=True),pd.DataFrame(results)], axis=1)
output.columns = ['comments','labels']

In [130]:
# Export the result to csv
output.to_csv('comment_results.csv', sep=',')

In [131]:
# The result of section assignment for unknown labels
output['labels'].value_counts()

Politics                      125002
Sunday Review                  58799
Asia Pacific                    2620
Live                            2458
Europe                          1180
Middle East                      914
Pro Football                     910
Retirement                       816
Eat                              780
Family                           735
Move                             402
DealBook                         401
Television                       319
Baseball                         308
Economy                          148
Energy & Environment             122
Olympics                         117
Mind                             100
College Basketball                77
Music                             56
Education Life                    52
Pro Basketball                    48
401(k)'s and Similar Plans        19
Book Review                       12
Room For Debate                    7
Lesson Plans                       7
Soccer                             6
A

In [153]:
output_pd = pd.DataFrame(output)
output_pd.columns = ['Category','Predict_count']
rows_count = output_pd.shape[0]
prediction_spread = output_pd['Predict_count'].value_counts()
prediction_spread = pd.DataFrame(prediction_spread)
rows_prediction_count = output_pd.shape[0]
def get_percentage_prediction(num):
    return float(num)/rows_prediction_count
prediction_spread['Percent_predict'] = prediction_spread.Predict_count.apply(get_percentage_prediction)
prediction_spread['Index'] = prediction_spread.index

In [154]:
results = pd.merge(training_spread, prediction_spread, on="Index")

In [150]:
# The report to show data imbalance, but is okay
results = results[['Index','Training_count','Percent_training','Predict_count','Percent_predict']]
results['Percent_predict_more'] = results['Percent_predict'] - results['Percent_training']
results

,Index,Training_count,Percent_training,Predict_count,Percent_predict,Percent_predict_more
0,Politics,75375,0.531679,125002,0.636395,0.104717
1,Sunday Review,25273,0.178270,58799,0.299350,0.121080
2,Middle East,7516,0.053016,914,0.004653,-0.048363
3,Asia Pacific,5719,0.040341,2620,0.013339,-0.027002
4,Europe,3825,0.026981,1180,0.006007,-0.020973
5,Television,2276,0.016054,319,0.001624,-0.014430
6,Family,2269,0.016005,735,0.003742,-0.012263
7,Economy,2129,0.015017,148,0.000753,-0.014264
8,Olympics,1955,0.013790,117,0.000596,-0.013194
9,Live,1174,0.008281,2458,0.012514,0.004233
